# Building Detector - Colab ML Server

This notebook provides the machine learning backend for the Building Detector application. It uses **SAM2 (Segment Anything Model 2)** to detect buildings from satellite imagery based on user-provided point coordinates.

## Overview
- **Model**: SAM2 Hiera Large for high-accuracy building detection
- **Input**: Satellite images + point coordinates from web interface
- **Output**: Regularized building polygons as GeoJSON
- **Deployment**: Ngrok tunnel for external access

## Requirements
- Google Colab with GPU runtime (recommended)
- Ngrok account with auth token
- Internet connection for model downloads (~600MB on first run)

## 1. Install Dependencies

Installing the required packages for the ML backend server.

In [ ]:
!pip install flask-ngrok flask-cors pyngrok
%pip install -U segment-geospatial

## 2. Configure Ngrok

Set up ngrok to create a public tunnel for external access to the Flask server.

⚠️ **Important**: Replace `'YOUR_AUTH_TOKEN'` with your actual ngrok token from [ngrok.com](https://ngrok.com)

In [ ]:
from pyngrok import ngrok, conf
import os

# Replace 'YOUR_AUTH_TOKEN' with your actual token from ngrok.com
NGROK_AUTH_TOKEN = 'YOUR_AUTH_TOKEN'
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Configure ngrok region (optional - use 'eu' for Europe)
conf.get_default().region = 'eu'

## 3. Building Detection Server

Flask API server that processes satellite images and detects buildings using SAM2.

### Processing Pipeline:
1. **Input Validation**: Check for image and points data
2. **SAM2 Initialization**: Load the model and set the image
3. **Point-based Prediction**: Use provided coordinates to guide detection
4. **Region Grouping**: Combine detected pixels into building shapes
5. **Regularization**: Clean and smooth building polygons
6. **Output**: Return GeoJSON with building footprints

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
from samgeo import SamGeo2, regularize
import tempfile
import json
import numpy as np
import rasterio

# Configuration
MODEL_ID = "sam2-hiera-large"  # Best accuracy model
MIN_REGION_SIZE = 200  # Minimum building size in pixels
FLASK_PORT = 5000

app = Flask(__name__)
CORS(app)

@app.route('/detect', methods=['POST'])
def detect():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image file provided'}), 400

        if 'points' not in request.form:
            return jsonify({'error': 'No points data provided'}), 400

        points = json.loads(request.form['points'])
        print(f"Received points: {points}")

        with tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_image:
            request.files['image'].save(temp_image.name)
            image_path = temp_image.name
            print(f"Saved image to: {image_path}")

        # Initialize SAM2 model
        sam = SamGeo2(model_id=MODEL_ID, automatic=False)
        sam.set_image(image_path)
        print("SAM model initialized and image set")

        with tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_mask, \
            tempfile.NamedTemporaryFile(suffix='.geojson', delete=False) as temp_vector, \
            tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_buildings, \
            tempfile.NamedTemporaryFile(suffix='.geojson', delete=False) as output_regularized:

            print("Starting prediction with points...")
            sam.predict_by_points(
                point_coords_batch=points,
                point_crs="EPSG:4326",
                output=temp_mask.name,
                dtype="uint8",
            )
            print(f"Prediction completed, mask saved to: {temp_mask.name}")

            print("Starting region grouping...")
            array, gdf = sam.region_groups(
                temp_mask.name,
                min_size=MIN_REGION_SIZE,
                out_vector=temp_vector.name,
                out_image=temp_buildings.name
            )
            print(f"Region grouping completed, vector saved to: {temp_vector.name}")

            print("Starting shape regularization...")
            regularize(temp_vector.name, output_regularized.name)
            print(f"Regularization completed, saved to: {output_regularized.name}")

            with open(output_regularized.name, 'rb') as f:
                result = f.read()
            print("Result read successfully")

        # Cleanup temporary files
        os.unlink(image_path)
        os.unlink(temp_mask.name)
        os.unlink(temp_vector.name)
        os.unlink(temp_buildings.name)
        os.unlink(output_regularized.name)

        return result, 200, {'Content-Type': 'application/json'}

    except Exception as e:
        import traceback
        print(f"Error in detect: {str(e)}")
        print(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

## 4. Start Server

Create ngrok tunnel and start the Flask server.

📋 **Copy the Public URL** that appears below and paste it into your main app's `config.py` file.

In [ ]:
# Start ngrok tunnel
public_url = ngrok.connect(FLASK_PORT)
print(f'🌐 Public URL: {public_url}')
print('📋 Copy this URL to your main app\'s config.py file')
print('🚀 Starting Flask server...')

# Start Flask server
app.run(port=FLASK_PORT)